<a href="https://colab.research.google.com/github/fzanart/GHDomains/blob/main/Bert_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpletransformers

In [2]:
from simpletransformers.classification import ClassificationModel
import torch
import numpy as np


In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(use_cuda, device)

True cuda


In [4]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
df = pd.read_csv('/content/drive/MyDrive/GHDomains/expanded_frame_final.csv')

In [6]:
df = df[['Description', 'Domain']]
df

,Description,Domain
0,freeCodeCamp.org's open source codebase and cu...,Documentation
1,"The most popular HTML, CSS, and JavaScript fra...",Web libraries and frameworks
2,"Bring data to life with SVG, Canvas and HTML. ...",Web libraries and frameworks
3,"A declarative, efficient, and flexible JavaScr...",Web libraries and frameworks
4,AngularJS - HTML enhanced for web apps!,Web libraries and frameworks
...,...,...
4995,This project aims to provide a working page fl...,Non-web libraries and frameworks
4996,JavaScript Client-Side Cookie Manipulation Lib...,Web libraries and frameworks
4997,The earliest versions of the very first c comp...,Software tools
4998,A clone of the UIImagePickerController using t...,Non-web libraries and frameworks


In [7]:
df['Domain'].unique()

array(['Documentation', 'Web libraries and frameworks', 'Software tools',
       'System software', 'Non-web libraries and frameworks',
       'Application software'], dtype=object)

In [8]:
labels = {'Application software':0,
          'Documentation':1,
          'Non-web libraries and frameworks':2,
          'Software tools':3,
          'System software':4,
          'Web libraries and frameworks':5
          }

In [9]:
df = df.replace({'Domain':labels})

In [10]:
df = df.rename(columns={"Description": "text", "Domain": "labels"})

In [11]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

4000 500 500


In [12]:
df_train

,text,labels
1501,A tool to automatically fix PHP Coding Standar...,3
2586,pagePiling plugin by Alvaro Trigo. Create a sc...,5
2653,The plugin manager for zsh.,3
1055,Rails for API only applications,2
705,Open source rich text editor based on HTML5 an...,5
...,...,...
3335,NO LONGER MAINTAINED. DEVELOP FOR 4.X+ DUDE.,2
1920,The Guardian DotCom.,0
3715,Zcash - Internet Money,0
4646,[OpenUDID IS NOW DEPRECATED] Open source initi...,2


In [15]:
weights = df['labels'].value_counts(normalize=True).sort_index(ascending=True).to_list()

In [16]:
model_args = {
    "num_train_epochs": 5,
    "learning_rate": 1e-4}



model = ClassificationModel("bert", "bert-base-cased", num_labels=6, weight = weights, use_cuda = True, args=model_args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [17]:
# Create a ClassificationModel
model.train_model(df_train)

  0%|          | 0/4000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/500 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Epoch 1 of 5:   0%|          | 0/500 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/500 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/500 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/500 [00:00<?, ?it/s]

(2500, 0.7357868715679273)

In [18]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(df_val)

  0%|          | 0/500 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/63 [00:00<?, ?it/s]

In [19]:
result

{'mcc': 0.526437742124292, 'eval_loss': 1.459717322732248}

In [20]:
model_outputs

array([[-0.53222656, -1.16699219, -1.0078125 , -0.6953125 , -2.14257812,
         4.46875   ],
       [ 0.74609375, -0.796875  , -0.91748047,  0.59179688, -1.55957031,
         1.5859375 ],
       [-2.0078125 , -1.5859375 ,  6.42578125, -1.45996094, -0.23718262,
        -0.63574219],
       ...,
       [-1.30957031, -1.4453125 , -1.06054688, -1.18164062, -2.19921875,
         5.9765625 ],
       [-1.69628906, -1.49902344, -0.53076172, -1.76660156, -2.26171875,
         6.2421875 ],
       [-1.74804688, -1.68359375, -0.76708984, -1.88085938, -2.0546875 ,
         6.34375   ]])

In [29]:
print('accuracy: ',(len(df_val) - len(wrong_predictions))/len(df_val))

accuracy:  0.642
